**1 处理前的准备**

1.1 导入必要的库

In [203]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from pathlib import Path
import pickle
import re
import os
import json


1.2 设置显示选项

In [179]:
pd.set_option('display.max_columns', None)  # 显示所有列
pd.set_option('display.max_rows', 50)      # 显示最多 50行

1.3 读取保存的 cleaned_data 字典

In [180]:
input_path = Path('../data/interim/cleaned_data.pkl')

if input_path.exists():
    with open(input_path, 'rb') as f:
        cleaned_data = pickle.load(f)
    print("✅ 成功加载 cleaned_data")
else:
    raise FileNotFoundError(f"未找到 cleaned_data.pkl，请先运行 02-cleaning.ipynb 并保存数据")

✅ 成功加载 cleaned_data


1.4 查看清洗后的数据

In [181]:
for sheet_name, df in cleaned_data.items():
    print(f"\n【{sheet_name}】前5行数据：")
    display(df.head())


【在玩】前5行数据：


,标题,简介,豆瓣评分,创建时间,我的评分,类型,发售时间
0,巫师3：狂猎 The Witcher 3: Wild Hunt,The Witcher III: Wild Hunt/猎魔人3：狂猎/角色扮演/冒险/动作/...,9.5,2025-05-06 16:11:28,5,"[角色扮演, 冒险, 动作]",2015-05-19
1,荒野大镖客：救赎2 Red Dead Redemption 2,碧血狂杀2/碧血狂殺2/血色救赎2/血色救贖2/Red Dead Redemption II...,9.7,2025-05-06 16:10:36,5,"[射击, 冒险, 动作]",2018-10-26



【想玩】前5行数据：


,标题,简介,豆瓣评分,创建时间,类型,发售时间
0,博德之门3 Baldur's Gate 3,角色扮演/冒险/策略/PC/Mac/PlayStation 5/Xbox Series X/...,9.6,2025-05-01 19:39:15,"[角色扮演, 冒险, 策略]",2023-08-03
1,苏丹的游戏,Sultan's Game/スルタンのゲーム/卡牌/模拟/角色扮演/策略/PC/2025-0...,暂无评分,2025-04-25 21:57:26,"[模拟, 角色扮演, 策略]",2025-03-31
2,光与影：33号远征队 Clair Obscur: Expedition 33,光與影：33號遠征隊/클레르 옵스퀴르: 33 원정대/角色扮演/冒险/动作/PC/Play...,暂无评分,2025-04-25 21:56:54,"[角色扮演, 冒险, 动作]",2025-04-24



【想看】前5行数据：


...

,标题,简介,豆瓣评分,创建时间,我的评分,类型,发售时间
0,山海旅人 The Rewinder,横版过关/角色扮演/冒险/PC/Mac/Xbox Series X/Nintendo Swi...,8.7,2025-05-06 16:14:28,4,"[横版过关, 角色扮演, 冒险]",2021-09-10
1,完蛋！我被美女包围了！,Love Is All Around/文字冒险/PC/Mac/iPhone/Android/...,7.7,2025-05-06 16:14:09,5,[文字冒险],2023-10-18
2,艾尔登法环 Elden Ring,艾爾登法環/老头环/上古之环/エルデンリング/角色扮演/冒险/动作/PC/PlayStati...,9.4,2025-05-06 16:13:05,5,"[角色扮演, 冒险, 动作]",2022-02-25
3,求生之路2 Left 4 Dead 2,L4D2/惡靈勢力 2/第一人称射击/射击/冒险/动作/PC/Mac/Linux/Xbox ...,9.0,2025-05-06 16:09:56,4,"[射击, 冒险, 动作]",2009-11-17
4,森林 The Forest,角色扮演/冒险/PC/PlayStation 4/Steam VR/Endnight Gam...,8.5,2025-05-06 16:09:12,4,"[角色扮演, 冒险]",2018-05-01



【看过】前5行数据：


,标题,简介,豆瓣评分,创建时间,我的评分,年份,国家/地区,类型,导演,主演
1,惊天魔盗团,2013 / 美国 法国 / 剧情 悬疑 犯罪 / 路易斯·莱特里尔 / 杰西·艾森伯格 艾...,7.8,2025-05-03 12:54:07,4,2013,"[美国, 法国]","[剧情, 悬疑, 犯罪]",[路易斯·莱特里尔],"[杰西·艾森伯格, 艾拉·菲舍尔]"
2,唐探1900,2025 / 中国大陆 中国香港 / 喜剧 动作 悬疑 / 陈思诚 戴墨 / 王宝强 刘昊然,6.5,2025-05-02 14:00:17,4,2025,"[中国大陆, 中国香港]","[喜剧, 动作, 悬疑]","[陈思诚, 戴墨]","[王宝强, 刘昊然]"
3,恶缘,2025 / 韩国 / 剧情 惊悚 犯罪 / 李日炯 / 朴海秀 申敏儿,8.4,2025-05-02 11:58:51,4,2025,[韩国],"[剧情, 惊悚, 犯罪]",[李日炯],"[朴海秀, 申敏儿]"
4,穿越火线,2020 / 中国大陆 / 剧情 奇幻 / 许宏宇 / 鹿晗 吴磊,7.9,2025-05-01 23:29:51,5,2020,[中国大陆],"[剧情, 奇幻]",[许宏宇],"[鹿晗, 吴磊]"
5,好小子们,2019 / 美国 / 喜剧 儿童 冒险 / 吉恩·斯图普尼兹基 / 雅各布·特伦布莱 基思...,7.2,2025-05-01 23:29:15,5,2019,[美国],"[喜剧, 儿童, 冒险]",[吉恩·斯图普尼兹基],"[雅各布·特伦布莱, 基思·L·威廉姆斯]"



【读过】前5行数据：


,标题,简介,豆瓣评分,创建时间,我的评分,作者,出版年份,出版社
0,13 67,陳浩基 / 2014 / 皇冠文化出版有限公司,9.1,2025-05-06 20:52:13,5,[陳浩基],2014,皇冠文化出版有限公司
1,无人生还,[英] 阿加莎・克里斯蒂 / 2008 / 人民文学出版社,9.0,2025-05-06 20:46:27,5,[阿加莎・克里斯蒂],2008,人民文学出版社
2,城南旧事,林海音 文 关维兴 图 / 2003 / 中国青年出版社,9.1,2025-05-06 20:46:05,5,"[林海音, 关维兴]",2003,中国青年出版社
3,东方快车谋杀案,[英] 阿加莎·克里斯蒂 / 2006 / 人民文学出版社,9.1,2025-05-06 20:45:53,5,[阿加莎·克里斯蒂],2006,人民文学出版社
4,麦田里的守望者,[美国] J. D. 塞林格 / 2007 / 译林出版社,8.1,2025-05-05 13:46:23,5,[塞林格],2007,译林出版社


1.5 创建输出目录

In [189]:
# 确保输出目录存在
base_output_dir = Path("../data/eda_results")
try:
    os.makedirs(base_output_dir, exist_ok=True)
    print(f"创建或确认 {base_output_dir} 目录成功")
except Exception as e:
    print(f"创建 {base_output_dir} 目录失败: {e}")
    exit()

创建或确认 ..\data\eda_results 目录成功


**2 影视作品分析**

In [190]:
def analyze_movie_sheet(df, sheet_name="合并数据", is_watched=False):
    """
    分析单个影视 sheet 的统计信息，并返回结果字典
    参数：
        df: DataFrame，包含影视数据
        sheet_name: str，sheet 名称（用于输出和保存）
        is_watched: bool，是否为“看过”sheet（控制“我的评分”分析）
    返回：
        dict：包含所有分析结果
    """
    print(f"\n📊 分析 {sheet_name} 📊")
    results = {"sheet_name": sheet_name}

    # 1. 作品数量
    total = len(df)
    print(f"1. 作品数量: {total}")
    results["total_count"] = total

    # 2. 豆瓣评分统计
    rating_stats = {}
    if '豆瓣评分' in df.columns:
        valid_ratings = df[df['豆瓣评分'].apply(lambda x: isinstance(x, (int, float)))]['豆瓣评分']
        if not valid_ratings.empty:
            rating_stats = {
                "valid_count": len(valid_ratings),
                "min": float(valid_ratings.min()),
                "max": float(valid_ratings.max()),
                "mean": float(valid_ratings.mean()),
                "median": float(valid_ratings.median())
            }
            print(f"\n2. 豆瓣评分统计（有效评分数: {len(valid_ratings)}）:")
            print(f"   - 最低评分: {valid_ratings.min():.1f}")
            print(f"   - 最高评分: {valid_ratings.max():.1f}")
            print(f"   - 平均评分: {valid_ratings.mean():.2f}")
            print(f"   - 中位评分: {valid_ratings.median():.2f}")
        else:
            print("\n2. 数据提示：无有效豆瓣评分数据")
            rating_stats["note"] = "无有效豆瓣评分数据"
    else:
        print("\n2. 数据提示：未找到‘豆瓣评分’字段")
        rating_stats["note"] = "未找到‘豆瓣评分’字段"
    results["douban_rating"] = rating_stats

    # 3. 我的评分统计
    my_rating_stats = {}
    if is_watched and '我的评分' in df.columns:
        valid_my_ratings = pd.to_numeric(df['我的评分'], errors='coerce').dropna()
        if not valid_my_ratings.empty:
            mean_my_rating = valid_my_ratings.mean() * 2
            my_rating_stats["mean_10_scale"] = float(mean_my_rating)
            print(f"\n3. 我的评分统计（10分制，转换自5分制）:")
            print(f"   - 平均评分: {mean_my_rating:.2f}")
        else:
            print("\n3. 数据提示：无有效‘我的评分’数据")
            my_rating_stats["note"] = "无有效‘我的评分’数据"
    else:
        print("\n3. 我的评分-不适用")
        my_rating_stats["note"] = "我的评分-不适用"
    results["my_rating"] = my_rating_stats

    # 4. 年份统计
    year_stats = {}
    if '年份' in df.columns:
        valid_years = df[df['年份'].apply(lambda x: x != '未知' and pd.notnull(x))]['年份'].astype(int)
        if not valid_years.empty:
            year_stats = {
                "min": int(valid_years.min()),
                "max": int(valid_years.max()),
                "median": int(valid_years.median())
            }
            print(f"\n4. 年份统计:")
            print(f"   - 最早年份: {valid_years.min()}")
            print(f"   - 最新年份: {valid_years.max()}")
            print(f"   - 中位年份: {int(valid_years.median())}")
        else:
            print("\n4. 数据提示：无有效年份数据")
            year_stats["note"] = "无有效年份数据"
    else:
        print("\n4. 数据提示：未找到‘年份’字段")
        year_stats["note"] = "未找到‘年份’字段"
    results["year"] = year_stats

    # 5. 创建时间按季度
    time_stats = {}
    if '创建时间' in df.columns:
        df['创建时间'] = pd.to_datetime(df['创建时间'], errors='coerce')
        valid_times = df[df['创建时间'].notnull()]
        if not valid_times.empty:
            valid_times['年份-季度'] = valid_times['创建时间'].dt.to_period('Q')
            time_counts = valid_times.groupby('年份-季度').size().sort_index()
            # 转换为 dict 以确保 JSON 可序列化
            time_stats = {str(k): int(v) for k, v in time_counts.items()}
            print(f"\n5. 创建时间按季度统计:")
            print(time_counts)
        else:
            print("\n5. 数据提示：无有效创建时间数据")
            time_stats["note"] = "无有效创建时间数据"
    else:
        print("\n5. 数据提示：未找到‘创建时间’字段")
        time_stats["note"] = "未找到‘创建时间’字段"
    results["create_time"] = time_stats

    # 6. 国家/地区统计
    country_stats = {}
    if '国家/地区' in df.columns:
        df['国家/地区'] = df['国家/地区'].apply(lambda x: x if isinstance(x, list) else [])
        all_countries = [c for countries in df['国家/地区'] for c in countries]
        if all_countries:
            country_counter = Counter(all_countries)
            top_5_countries = country_counter.most_common(5)
            country_stats["top_5"] = [{"country": c, "count": int(cnt)} for c, cnt in top_5_countries]
            country_stats["full_counts"] = {k: int(v) for k, v in country_counter.items()}
            print(f"\n6. 国家/地区 Top 5:")
            for country, count in top_5_countries:
                print(f"   - {country}: {count} 次")
        else:
            print("\n6. 数据提示：无国家/地区数据")
            country_stats["note"] = "无国家/地区数据"
    else:
        print("\n6. 数据提示：未找到‘国家/地区’字段")
        country_stats["note"] = "未找到‘国家/地区’字段"
    results["country"] = country_stats

    # 7. 导演统计
    director_stats = {}
    if '导演' in df.columns:
        df['导演'] = df['导演'].apply(lambda x: x if isinstance(x, list) else [])
        director_list = [d for directors in df['导演'] for d in directors]
        if director_list:
            director_counter = Counter(director_list)
            top_5_directors = director_counter.most_common(5)
            director_stats["top_5"] = [{"director": d, "count": int(cnt)} for d, cnt in top_5_directors]
            director_stats["full_counts"] = {k: int(v) for k, v in director_counter.items()}
            print(f"\n7. 导演 Top 5:")
            for director, count in top_5_directors:
                print(f"   - {director}: {count} 次")
        else:
            print("\n7. 数据提示：无导演数据")
            director_stats["note"] = "无导演数据"
    else:
        print("\n7. 数据提示：未找到‘导演’字段")
        director_stats["note"] = "未找到‘导演’字段"
    results["director"] = director_stats

    # 8. 演员统计
    actor_stats = {}
    if '主演' in df.columns:
        df['主演'] = df['主演'].apply(lambda x: x if isinstance(x, list) else [])
        actor_list = [a for actors in df['主演'] for a in actors]
        if actor_list:
            actor_counter = Counter(actor_list)
            top_5_actors = actor_counter.most_common(5)
            actor_stats["top_5"] = [{"actor": a, "count": int(cnt)} for a, cnt in top_5_actors]
            actor_stats["full_counts"] = {k: int(v) for k, v in actor_counter.items()}
            print(f"\n8. 演员 Top 5:")
            for actor, count in top_5_actors:
                print(f"   - {actor}: {count} 次")
        else:
            print("\n8. 数据提示：无演员数据")
            actor_stats["note"] = "无演员数据"
    else:
        print("\n8. 数据提示：未找到‘主演’字段")
        actor_stats["note"] = "未找到‘主演’字段"
    results["actor"] = actor_stats

    # 9. 影视类型 Top 5
    genre_stats = {}
    if '类型' in df.columns:
        df['类型'] = df['类型'].apply(lambda x: x if isinstance(x, list) else [])
        all_genres = [g for genres in df['类型'] for g in genres]
        if all_genres:
            genre_counter = Counter(all_genres)
            top_5_genres = genre_counter.most_common(5)
            genre_stats["top_5"] = [{"genre": g, "count": int(cnt)} for g, cnt in top_5_genres]
            genre_stats["full_counts"] = {k: int(v) for k, v in genre_counter.items()}
            print(f"\n9. 影视类型 Top 5:")
            for genre, count in top_5_genres:
                print(f"   - {genre}: {count} 次")
        else:
            print("\n9. 数据提示：无影视类型数据")
            genre_stats["note"] = "无影视类型数据"
    else:
        print("\n9. 数据提示：未找到‘类型’字段")
        genre_stats["note"] = "未找到‘类型’字段"
    results["genre"] = genre_stats

    return results

In [196]:
# 定义影视作品 sheet 名称
movie_sheets = ['想看', '在看', '看过']

# 检查是否有有效 sheet
movie_sheets = [sheet for sheet in movie_sheets if sheet in cleaned_data]
if not movie_sheets:
    print("数据提示：未找到任何影视作品数据")
    exit()

print("🎬 开始对影视作品进行分项统计分析...\n")

# 单独分析每个 sheet 并收集结果
all_results = []
for sheet in movie_sheets:
    is_watched = (sheet == '看过')
    result = analyze_movie_sheet(cleaned_data[sheet], sheet_name=sheet, is_watched=is_watched)
    all_results.append(result)

# 合并所有影视作品数据
combined_df = pd.concat([cleaned_data[sheet] for sheet in movie_sheets], ignore_index=True)

# 合并数据分析
print("\n🎬 合并所有影视作品数据分析 🎬")
result = analyze_movie_sheet(combined_df, sheet_name="所有影视作品", is_watched=True)
all_results.append(result)

# 保存汇总统计到 JSON
json_path = base_output_dir / "summary_movies_results.json"
try:
    # 调试：打印 all_results 的长度和内容概要
    print(f"\n调试：准备保存 {len(all_results)} 个分析结果到 JSON")
    for res in all_results:
        print(f"  - Sheet: {res['sheet_name']}, 作品数量: {res['total_count']}")
    
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2)
    print(f"保存汇总统计到 {json_path}")
except Exception as e:
    print(f"保存汇总统计失败: {e}")

print("\n🎥 分析完成！")

🎬 开始对影视作品进行分项统计分析...


📊 分析 想看 📊
1. 作品数量: 73

2. 豆瓣评分统计（有效评分数: 73）:
   - 最低评分: 4.9
   - 最高评分: 9.7
   - 平均评分: 8.39
   - 中位评分: 8.70

3. 我的评分-不适用

4. 年份统计:
   - 最早年份: 1988
   - 最新年份: 2025
   - 中位年份: 2018

5. 创建时间按季度统计:
年份-季度
2021Q1     1
2021Q3     1
2021Q4     5
2022Q1    26
2022Q3     3
2022Q4    10
2023Q1     4
2023Q2     4
2023Q3     2
2023Q4     4
2024Q1     1
2024Q2     2
2024Q3     1
2024Q4     1
2025Q1     3
2025Q2     5
Freq: Q-DEC, dtype: int64

6. 国家/地区 Top 5:
   - 美国: 28 次
   - 日本: 17 次
   - 中国大陆: 15 次
   - 英国: 6 次
   - 韩国: 4 次

7. 导演 Top 5:
   - 凯文·布瑞: 2 次
   - 土井裕泰: 2 次
   - 金子文纪: 2 次
   - 高畑勋: 2 次
   - 克雷格·卓贝: 1 次

8. 演员 Top 5:
   - 松隆子: 3 次
   - 满岛光: 2 次
   - 堺雅人: 2 次
   - 新垣结衣: 2 次
   - 亚伦·保尔: 2 次

9. 影视类型 Top 5:
   - 剧情: 47 次
   - 喜剧: 24 次
   - 爱情: 16 次
   - 动画: 14 次
   - 犯罪: 13 次

📊 分析 看过 📊
1. 作品数量: 212

2. 豆瓣评分统计（有效评分数: 212）:
   - 最低评分: 2.1
   - 最高评分: 9.7
   - 平均评分: 8.03
   - 中位评分: 8.20

3. 我的评分统计（10分制，转换自5分制）:
   - 平均评分: 8.74

4. 年份统计:
   - 最早年份: 1994
   - 最新年份: 2025


**3 图书分析**

In [197]:
def analyze_book_sheet(df, sheet_name="合并数据", is_read=False):
    """
    分析单个图书 sheet 的统计信息，并返回结果字典
    参数：
        df: DataFrame，包含图书数据
        sheet_name: str，sheet 名称（用于输出和保存）
        is_read: bool，是否为“读过”sheet（控制“我的评分”分析）
    返回：
        dict：包含所有分析结果
    """
    print(f"\n📚 分析 {sheet_name} 📚")
    results = {"sheet_name": sheet_name}

    # 1. 图书数量
    total = len(df)
    print(f"1. 图书数量: {total}")
    results["total_count"] = total

    # 2. 豆瓣评分统计
    rating_stats = {}
    if '豆瓣评分' in df.columns:
        valid_ratings = df[df['豆瓣评分'].apply(lambda x: isinstance(x, (int, float)))]['豆瓣评分']
        if not valid_ratings.empty:
            rating_stats = {
                "valid_count": len(valid_ratings),
                "min": float(valid_ratings.min()),
                "max": float(valid_ratings.max()),
                "mean": float(valid_ratings.mean()),
                "median": float(valid_ratings.median())
            }
            print(f"\n2. 豆瓣评分统计（有效评分数: {len(valid_ratings)}）:")
            print(f"   - 最低评分: {valid_ratings.min():.1f}")
            print(f"   - 最高评分: {valid_ratings.max():.1f}")
            print(f"   - 平均评分: {valid_ratings.mean():.2f}")
            print(f"   - 中位评分: {valid_ratings.median():.2f}")
        else:
            print("\n2. 数据提示：无有效豆瓣评分数据")
            rating_stats["note"] = "无有效豆瓣评分数据"
    else:
        print("\n2. 数据提示：未找到‘豆瓣评分’字段")
        rating_stats["note"] = "未找到‘豆瓣评分’字段"
    results["douban_rating"] = rating_stats

    # 3. 我的评分统计
    my_rating_stats = {}
    if is_read and '我的评分' in df.columns:
        valid_my_ratings = pd.to_numeric(df['我的评分'], errors='coerce').dropna()
        if not valid_my_ratings.empty:
            mean_my_rating = valid_my_ratings.mean() * 2
            my_rating_stats["mean_10_scale"] = float(mean_my_rating)
            print(f"\n3. 我的评分统计（10分制，转换自5分制）:")
            print(f"   - 平均评分: {mean_my_rating:.2f}")
        else:
            print("\n3. 数据提示：无有效‘我的评分’数据")
            my_rating_stats["note"] = "无有效‘我的评分’数据"
    else:
        print("\n3. 我的评分-不适用")
        my_rating_stats["note"] = "我的评分-不适用"
    results["my_rating"] = my_rating_stats

    # 4. 出版年份统计
    year_stats = {}
    if '出版年份' in df.columns:
        valid_years = df[df['出版年份'].apply(lambda x: pd.notnull(x) and str(x).isdigit())]['出版年份'].astype(int)
        if not valid_years.empty:
            year_stats = {
                "min": int(valid_years.min()),
                "max": int(valid_years.max()),
                "median": int(valid_years.median())
            }
            print(f"\n4. 出版年份统计:")
            print(f"   - 最老年份: {valid_years.min()}")
            print(f"   - 最新年份: {valid_years.max()}")
            print(f"   - 中位年份: {int(valid_years.median())}")
        else:
            print("\n4. 数据提示：无有效出版年份数据")
            year_stats["note"] = "无有效出版年份数据"
    else:
        print("\n4. 数据提示：未找到‘出版年份’字段")
        year_stats["note"] = "未找到‘出版年份’字段"
    results["publish_year"] = year_stats

    # 5. 创建时间按季度
    time_stats = {}
    if '创建时间' in df.columns:
        df['创建时间'] = pd.to_datetime(df['创建时间'], errors='coerce')
        valid_times = df[df['创建时间'].notnull()]
        if not valid_times.empty:
            valid_times['年份-季度'] = valid_times['创建时间'].dt.to_period('Q')
            time_counts = valid_times.groupby('年份-季度').size().sort_index()
            time_stats = {str(k): int(v) for k, v in time_counts.items()}
            print(f"\n5. 创建时间按季度统计:")
            print(time_counts)
        else:
            print("\n5. 数据提示：无有效创建时间数据")
            time_stats["note"] = "无有效创建时间数据"
    else:
        print("\n5. 数据提示：未找到‘创建时间’字段")
        time_stats["note"] = "未找到‘创建时间’字段"
    results["create_time"] = time_stats

    # 6. 作者统计
    author_stats = {}
    if '作者' in df.columns:
        df['作者'] = df['作者'].apply(lambda x: x if isinstance(x, list) else [])
        author_list = [a for authors in df['作者'] for a in authors]
        if author_list:
            author_counter = Counter(author_list)
            top_5_authors = author_counter.most_common(5)
            author_stats["top_5"] = [{"author": a, "count": int(cnt)} for a, cnt in top_5_authors]
            author_stats["full_counts"] = {k: int(v) for k, v in author_counter.items()}
            print(f"\n6. 作者 Top 5:")
            for author, count in top_5_authors:
                print(f"   - {author}: {count} 次")
        else:
            print("\n6. 数据提示：无作者数据")
            author_stats["note"] = "无作者数据"
    else:
        print("\n6. 数据提示：未找到‘作者’字段")
        author_stats["note"] = "未找到‘作者’字段"
    results["author"] = author_stats

    # 7. 出版社统计
    publisher_stats = {}
    if '出版社' in df.columns:
        valid_publishers = df[df['出版社'].notnull()]['出版社']
        if not valid_publishers.empty:
            publisher_counter = Counter(valid_publishers)
            top_5_publishers = publisher_counter.most_common(5)
            publisher_stats["top_5"] = [{"publisher": p, "count": int(cnt)} for p, cnt in top_5_publishers]
            publisher_stats["full_counts"] = {k: int(v) for k, v in publisher_counter.items()}
            print(f"\n7. 出版社 Top 5:")
            for publisher, count in top_5_publishers:
                print(f"   - {publisher}: {count} 次")
        else:
            print("\n7. 数据提示：无出版社数据")
            publisher_stats["note"] = "无出版社数据"
    else:
        print("\n7. 数据提示：未找到‘出版社’字段")
        publisher_stats["note"] = "未找到‘出版社’字段"
    results["publisher"] = publisher_stats

    return results

In [198]:
# 定义图书 sheet 名称
book_sheets = ['想读', '读过', '在读']

# 检查是否有有效 sheet
book_sheets = [sheet for sheet in book_sheets if sheet in cleaned_data]
if not book_sheets:
    print("数据提示：未找到任何图书数据")
    exit()

print("📖 开始对图书进行分项统计分析...\n")

# 单独分析每个 sheet 并收集结果
all_results = []
for sheet in book_sheets:
    is_read = (sheet == '读过')
    result = analyze_book_sheet(cleaned_data[sheet], sheet_name=sheet, is_read=is_read)
    all_results.append(result)

# 合并所有图书数据
combined_df = pd.concat([cleaned_data[sheet] for sheet in book_sheets], ignore_index=True)

# 合并数据分析
print("\n📖 合并所有图书数据分析 📖")
result = analyze_book_sheet(combined_df, sheet_name="所有图书", is_read=True)
all_results.append(result)

# 保存汇总统计到 JSON
json_path = base_output_dir / "summary_books_results.json"
try:
    # 调试：打印 all_results 的长度和内容概要
    print(f"\n调试：准备保存 {len(all_results)} 个分析结果到 JSON")
    for res in all_results:
        print(f"  - Sheet: {res['sheet_name']}, 图书数量: {res['total_count']}")
    
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2)
    print(f"保存汇总统计到 {json_path}")
except Exception as e:
    print(f"保存汇总统计失败: {str(e)}")
    # 调试：打印 all_results 的部分内容以排查问题
    print("调试：all_results 示例内容：")
    print(json.dumps(all_results[:1], ensure_ascii=False, indent=2))

print("\n📚 分析完成！")

📖 开始对图书进行分项统计分析...


📚 分析 想读 📚
1. 图书数量: 155

2. 豆瓣评分统计（有效评分数: 155）:
   - 最低评分: 6.0
   - 最高评分: 9.8
   - 平均评分: 8.33
   - 中位评分: 8.50

3. 我的评分-不适用

4. 出版年份统计:
   - 最老年份: 1991
   - 最新年份: 2025
   - 中位年份: 2019

5. 创建时间按季度统计:
年份-季度
2021Q4     1
2022Q2     9
2022Q3     3
2022Q4     6
2023Q1     2
2023Q2    18
2023Q3     6
2023Q4     5
2024Q1    14
2024Q2    44
2024Q3    10
2024Q4    16
2025Q1    11
2025Q2    10
Freq: Q-DEC, dtype: int64

6. 作者 Top 5:
   - Sussman: 2 次
   - Robin: 2 次
   - Williams: 2 次
   - 李奇霖: 2 次
   - 瑞·达利欧: 2 次

7. 出版社 Top 5:
   - 机械工业出版社: 16 次
   - 中信出版社: 13 次
   - 人民邮电出版社: 8 次
   - 中国人民大学出版社: 8 次
   - 中信出版集团: 7 次

📚 分析 读过 📚
1. 图书数量: 34

2. 豆瓣评分统计（有效评分数: 34）:
   - 最低评分: 5.4
   - 最高评分: 9.6
   - 平均评分: 8.59
   - 中位评分: 8.70

3. 我的评分统计（10分制，转换自5分制）:
   - 平均评分: 8.94

4. 出版年份统计:
   - 最老年份: 2003
   - 最新年份: 2023
   - 中位年份: 2018

5. 创建时间按季度统计:
年份-季度
2022Q3    5
2023Q2    5
2023Q3    1
2023Q4    1
2024Q1    1
2024Q2    5
2024Q3    4
2024Q4    2
2025Q1    4
2025Q2    6
Freq: Q-DEC, dt

**3 游戏分析**

In [201]:
def analyze_game_sheet(df, sheet_name="合并数据", is_played=False):
    """
    分析单个游戏 sheet 的统计信息，并返回结果字典
    参数：
        df: DataFrame，包含游戏数据
        sheet_name: str，sheet 名称（用于输出和保存）
        is_played: bool，是否为“玩过”sheet（控制“我的评分”分析）
    返回：
        dict：包含所有分析结果
    """
    print(f"\n🎮 分析 {sheet_name} 🎮")
    results = {"sheet_name": sheet_name}

    # 1. 游戏数量
    total = len(df)
    print(f"1. 游戏数量: {total}")
    results["total_count"] = total

    # 2. 豆瓣评分统计
    rating_stats = {}
    if '豆瓣评分' in df.columns:
        valid_ratings = df[df['豆瓣评分'].apply(lambda x: isinstance(x, (int, float)))]['豆瓣评分']
        if not valid_ratings.empty:
            rating_stats = {
                "valid_count": len(valid_ratings),
                "min": float(valid_ratings.min()),
                "max": float(valid_ratings.max()),
                "mean": float(valid_ratings.mean()),
                "median": float(valid_ratings.median())
            }
            print(f"\n2. 豆瓣评分统计（有效评分数: {len(valid_ratings)}）:")
            print(f"   - 最低评分: {valid_ratings.min():.1f}")
            print(f"   - 最高评分: {valid_ratings.max():.1f}")
            print(f"   - 平均评分: {valid_ratings.mean():.2f}")
            print(f"   - 中位评分: {valid_ratings.median():.2f}")
        else:
            print("\n2. 数据提示：无有效豆瓣评分数据")
            rating_stats["note"] = "无有效豆瓣评分数据"
    else:
        print("\n2. 数据提示：未找到‘豆瓣评分’字段")
        rating_stats["note"] = "未找到‘豆瓣评分’字段"
    results["douban_rating"] = rating_stats

    # 3. 我的评分统计（仅“玩过”）
    my_rating_stats = {}
    if is_played and '我的评分' in df.columns:
        valid_my_ratings = pd.to_numeric(df['我的评分'], errors='coerce').dropna()
        if not valid_my_ratings.empty:
            mean_my_rating = valid_my_ratings.mean() * 2
            my_rating_stats["mean_10_scale"] = float(mean_my_rating)
            print(f"\n3. 我的评分统计（10分制，转换自5分制）:")
            print(f"   - 平均评分: {mean_my_rating:.2f}")
        else:
            print("\n3. 数据提示：无有效‘我的评分’数据")
            my_rating_stats["note"] = "无有效‘我的评分’数据"
    else:
        print("\n3. 我的评分-不适用")
        my_rating_stats["note"] = "我的评分-不适用"
    results["my_rating"] = my_rating_stats

    # 4. 创建时间按年份
    create_time_stats = {}
    if '创建时间' in df.columns:
        df['创建时间'] = pd.to_datetime(df['创建时间'], errors='coerce')
        valid_times = df[df['创建时间'].notnull()]
        if not valid_times.empty:
            year_stats = valid_times['创建时间'].dt.year.value_counts().sort_index()
            create_time_stats = {str(k): int(v) for k, v in year_stats.items()}
            print(f"\n4. 创建时间按年份统计:")
            print(year_stats)
        else:
            print("\n4. 数据提示：无有效创建时间数据")
            create_time_stats["note"] = "无有效创建时间数据"
    else:
        print("\n4. 数据提示：未找到‘创建时间’字段")
        create_time_stats["note"] = "未找到‘创建时间’字段"
    results["create_time"] = create_time_stats

    # 5. 发售时间按年份
    release_year_stats = {}
    def extract_release_year(row):
        if '简介' in row and isinstance(row['简介'], str):
            match = re.search(r'\b(\d{4})\b', row['简介'])
            if match:
                return int(match.group(1))
        if '创建时间' in row and pd.notnull(row['创建时间']):
            return row['创建时间'].year
        return None

    df['发售年份'] = df.apply(extract_release_year, axis=1)
    valid_release_years = df[df['发售年份'].notnull()]['发售年份'].astype(int)
    if not valid_release_years.empty:
        release_year_counts = valid_release_years.value_counts().sort_index()
        release_year_stats = {str(k): int(v) for k, v in release_year_counts.items()}
        print(f"\n5. 发售时间按年份统计:")
        print(release_year_counts)
    else:
        print("\n5. 数据提示：无有效发售年份数据")
        release_year_stats["note"] = "无有效发售年份数据"
    results["release_year"] = release_year_stats

    # 6. 游戏类型 Top 5
    genre_stats = {}
    if '类型' in df.columns:
        df['类型'] = df['类型'].apply(lambda x: x if isinstance(x, list) else [])
        all_genres = [g for genres in df['类型'] for g in genres]
        if all_genres:
            genre_counter = Counter(all_genres)
            top_5_genres = genre_counter.most_common(5)
            genre_stats["top_5"] = [{"genre": g, "count": int(cnt)} for g, cnt in top_5_genres]
            genre_stats["full_counts"] = {k: int(v) for k, v in genre_counter.items()}
            print(f"\n6. 游戏类型 Top 5:")
            for genre, count in top_5_genres:
                print(f"   - {genre}: {count} 次")
        else:
            print("\n6. 数据提示：无游戏类型数据")
            genre_stats["note"] = "无游戏类型数据"
    else:
        print("\n6. 数据提示：未找到‘类型’字段")
        genre_stats["note"] = "未找到‘类型’字段"
    results["genre"] = genre_stats

    return results

In [202]:
# 定义游戏 sheet 名称
game_sheets = ['想玩', '在玩', '玩过']

# 检查是否有有效 sheet
game_sheets = [sheet for sheet in game_sheets if sheet in cleaned_data]
if not game_sheets:
    print("数据提示：未找到任何游戏数据")
    exit()

print("🎮 开始对游戏进行分项统计分析...\n")

# 单独分析每个 sheet 并收集结果
all_results = []
for sheet in game_sheets:
    is_played = (sheet == '玩过')
    result = analyze_game_sheet(cleaned_data[sheet], sheet_name=sheet, is_played=is_played)
    all_results.append(result)

# 合并所有游戏数据
combined_df = pd.concat([cleaned_data[sheet] for sheet in game_sheets], ignore_index=True)

# 合并数据分析
print("\n🎮 合并所有游戏数据分析 🎮")
result = analyze_game_sheet(combined_df, sheet_name="所有游戏", is_played=True)
all_results.append(result)

# 保存汇总统计到 JSON
json_path = base_output_dir / "summary_games_results.json"
try:
    # 调试：打印 all_results 的长度和内容概要
    print(f"\n调试：准备保存 {len(all_results)} 个分析结果到 JSON")
    for res in all_results:
        print(f"  - Sheet: {res['sheet_name']}, 游戏数量: {res['total_count']}")
    
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2)
    print(f"保存汇总统计到 {json_path}")
except Exception as e:
    print(f"保存汇总统计失败: {str(e)}")
    # 调试：打印 all_results 的部分内容以排查问题
    print("调试：all_results 示例内容：")
    print(json.dumps(all_results[:1], ensure_ascii=False, indent=2))

print("\n🎮 分析完成！")

🎮 开始对游戏进行分项统计分析...


🎮 分析 想玩 🎮
1. 游戏数量: 3

2. 豆瓣评分统计（有效评分数: 1）:
   - 最低评分: 9.6
   - 最高评分: 9.6
   - 平均评分: 9.60
   - 中位评分: 9.60

3. 我的评分-不适用

4. 创建时间按年份统计:
创建时间
2025    3
Name: count, dtype: int64

5. 发售时间按年份统计:
发售年份
2023    1
2025    2
Name: count, dtype: int64

6. 游戏类型 Top 5:
   - 角色扮演: 3 次
   - 冒险: 2 次
   - 策略: 2 次
   - 模拟: 1 次
   - 动作: 1 次

🎮 分析 在玩 🎮
1. 游戏数量: 2

2. 豆瓣评分统计（有效评分数: 2）:
   - 最低评分: 9.5
   - 最高评分: 9.7
   - 平均评分: 9.60
   - 中位评分: 9.60

3. 我的评分-不适用

4. 创建时间按年份统计:
创建时间
2025    2
Name: count, dtype: int64

5. 发售时间按年份统计:
发售年份
2015    1
2018    1
Name: count, dtype: int64

6. 游戏类型 Top 5:
   - 冒险: 2 次
   - 动作: 2 次
   - 角色扮演: 1 次
   - 射击: 1 次

🎮 分析 玩过 🎮
1. 游戏数量: 15

2. 豆瓣评分统计（有效评分数: 15）:
   - 最低评分: 6.5
   - 最高评分: 9.6
   - 平均评分: 8.70
   - 中位评分: 8.70

3. 我的评分统计（10分制，转换自5分制）:
   - 平均评分: 9.47

4. 创建时间按年份统计:
创建时间
2025    15
Name: count, dtype: int64

5. 发售时间按年份统计:
发售年份
2007    1
2009    1
2011    1
2014    2
2016    1
2018    1
2019    3
2021    1
2022    1
2023    1
2024    1
2077    1